# Run multiple robots for one day

Run an experiment with multiple robots exploring an environment during the course of one day. 

The specification of the experiment is fully determined by the exp/run of type nrobot1day. This exp/run will refer to other exp/runs to specify the environment, the estimator and the scoring function. It will also list the names of the robots, their policies and other information. 

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
from pprint import pprint

from wbf_simulate import run_nrobot1day

In [2]:
# the experiment for the robot run
experiment = "nrobot1day"
# run = "two-random-robots"
# run = "two-communicating-robots"
# run = "mrmr-3random"
# run = "mrmr-3lawnmowers"
# run = "mrmr-1pioneer-2contractors"
run = "mrmr-1pio-2con-clustered"
# exp = Config().get_experiment(experiment, run)
exp = Config().get_experiment(experiment, run, creation_style="discard-old")
pprint(exp)

***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/WaterBerryFarms/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/WaterBerryFarms/settings/WBF-config-szenes.yaml
***ExpRun**: Configuration for exp/run: nrobot1day/mrmr-1pio-2con-clustered successfully loaded
***ExpRun**: Removing existing experiment directory /Users/lboloni/Documents/Develop/Data/WBF/nrobot1day/mrmr-1pio-2con-clustered
Experiment:
    data_dir: /Users/lboloni/Documents/Develop/Data/WBF/nrobot1day/mrmr-1pio-2con-clustered
    exp_environment: environment
    exp_estimator: estimator
    exp_run_sys_indep_file: /Users/lboloni/Documents/Develop/Github/WBF/WaterberryFarms/experiment_configs/nrobot1day/mrmr-1pio-2con-clustered.yaml
    exp_score: score
    experiment_name: nrobot1day
    im_resolution: 10
    name: 1 pioneer 2 contractors clustered
    robots:
    - exp-policy-extra-parameters:
        budget: 1000
    

In [3]:
run_nrobot1day(exp)

***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/WBF/WaterberryFarms/experiment_configs/environment/_defaults_environment.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: environment/mrmr-custom-clustered successfully loaded
Experiment:
    custom-tylcv: mrmr-clustered-100.png
    data_dir: /Users/lboloni/Documents/Develop/Data/WBF/environment/mrmr-custom-clustered
    exp_run_sys_indep_file: /Users/lboloni/Documents/Develop/Github/WBF/WaterberryFarms/experiment_configs/environment/mrmr-custom-clustered.yaml
    experiment_name: environment
    precompute-time: 10
    run_name: mrmr-custom-clustered
    subrun_name: null
    typename: Miniberry-100

***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/WBF/WaterberryFarms/experiment_configs/estimator/_defaults_estimator.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: estimator/adaptive-disk successfully loaded
Experiment:
    data_dir: /Users/lboloni/Document

INFO:root:create_type_map shape=(10201,)
INFO:root:Infection matrix: [0.         0.25       0.4375     0.578125   0.68359375 0.76269531
 0.82202148 0.86651611 0.89988708 0.92491531 0.94368649 0.95776486
 0.96832365 0.97624274 0.98218205 0.98663654 0.9899774  0.99248305
 0.99436229 0.99577172 0.99682879 0.99762159 0.99821619 0.99866214
 0.99899661]
INFO:root:Infection matrix: [0.         0.15       0.2775     0.385875   0.47799375 0.55629469
 0.62285048 0.67942291 0.72750947 0.76838305 0.8031256  0.83265676
 0.85775824 0.87909451 0.89723033 0.91264578 0.92574891 0.93688658
 0.94635359 0.95440055 0.96124047 0.9670544  0.97199624 0.9761968
 0.97976728]
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 1
INFO:root:PrecalculatedEnvironment: file /Users/lboloni/Documents/Develop/Data/WBF/environment/mrmr-custom-clustered/precalc_tylcv/env_value_00001.gz already exists, skipping!


policy-name
policy-code
seed
budget
[{'exp-policy': Experiment:
    budget: 1000
    data_dir: /Users/lboloni/Documents/Develop/Data/WBF/policy/generic
    exp_run_sys_indep_file: /Users/lboloni/Documents/Develop/Github/WBF/WaterberryFarms/experiment_configs/policy/generic.yaml
    experiment_name: policy
    policy-code: MRMR_Pioneer
    policy-name: mrmr-pioneer
    run_name: generic
    seed: 1
    subrun_name: null
,
  'name': 'robot-pioneer'},
 {'exp-policy': Experiment:
    budget: 1000
    data_dir: /Users/lboloni/Documents/Develop/Data/WBF/policy/generic
    exp_run_sys_indep_file: /Users/lboloni/Documents/Develop/Github/WBF/WaterberryFarms/experiment_configs/policy/generic.yaml
    experiment_name: policy
    policy-code: MRMR_Contractor
    policy-name: mrmr-contractor-1
    run_name: generic
    seed: 2
    subrun_name: null
,
  'name': 'robot-contr-1'},
 {'exp-policy': Experiment:
    budget: 1000
    data_dir: /Users/lboloni/Documents/Develop/Data/WBF/policy/generic
    ex